# BSc Thesis: Evaluation of Decision Tree and Random Forest Classifiers in the Finance Domain

## Table of Contents
0. Preparation
1. Data Preparation Stage
2. Feature Extraction
3. Classification
4. Evaluation of Models
5. Visualisations

# 0 | Preparation

### Imports

In [294]:
# Data manipulation and arrays
import pandas as pd
import numpy as np

# Machine learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn import metrics

# Plottig
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import display

### Constant variables

In [335]:
# Define filenames for technology stock .CSV datasets
MSFT_DATA = "Datasets/Kaggle_SnP500_MSFT_2013-2018.csv"
AAPL_DATA = "Datasets/Kaggle_SnP500_AAPL_2013-2018.csv"
GOOGL_DATA = "Datasets/Kaggle_SnP500_GOOGL_2013-2018.csv"
HP_DATA = "Datasets/Kaggle_SnP500_HP_2013-2018.csv"
IBM_DATA = "Datasets/Kaggle_SnP500_IBM_2013-2018.csv"
WU_DATA = "Datasets/Kaggle_SnP500_WU_2013-2018.csv"
XRX_DATA = "Datasets/Kaggle_SnP500_XRX_2013-2018.csv"
TECH_GROUP = [MSFT_DATA, AAPL_DATA, GOOGL_DATA, HP_DATA, IBM_DATA, WU_DATA, XRX_DATA] # this list will be used to iterate over all datasets

# Define time horizons to compare classification results for 1-day to 1-year predictions
TIME_HORIZONS = [1, 5, 10, 30, 100, 365]

# Define verbosity
VERBOSE = False

# Centrally define if figures should be saved to ./plots/
SAVE_FIG = False

# 1 | Data Preparation Stage
- Load data and adjust columns as needed
- Extract features for technical analysis
- Define class for later classification
- Detect anomalies in the datasets
- No feature selection needed as embedded in Decision Trees (DT) and Random Forests (RF)

## 1.1 | Load Datasets
- For an apples-to-apples comparison, technology companies are analyzed (idea: companies/stocks within an industry have similar drivers)
- Selected stocks differ in price trends (upward- vs constant- vs downward trend)

In [352]:
def load_OHLC_data(filename=MSFT_DATA, time_horizons=TIME_HORIZONS, verbose=VERBOSE, save_fig=SAVE_FIG):
    """
    Loads basic stock data (date, name, open, high, low, close) from a given .CSV file and returns a corresponding DataFrame.
    Unnecessary categorical columns are dropped, and necessary columns (e.g. month as number) are added.
    """
    try:
        df = pd.read_csv(filename)
        
        if save_fig is True:
            # Visualize loaded time series data
            df["date"] = pd.to_datetime(df["date"])
            df.plot(x="date", y="close", figsize=(12,6), legend=None)
            plt.xlabel("Time [Year]")
            plt.ylabel("Price [daily closing price in USD]")
            plt.title(df["Name"][0] + "-Stock Data 2013 to 2018");
            plt.savefig("./Plots/" + df["Name"][0] + "-Stock-Price-Plot.jpeg")
        
        # Calculate base column for later class: future return of stock over given time horizon (e.g. this week's Monday to next week's Monday)
        for horizon in time_horizons:
            df["return_future_" + str(horizon) + "d"] = (df["close"].shift(-1*horizon)/df["close"])-1
        
        # Convert date to numerical month to possibly detect cyclicality (e.g. christmas effect) in time series
        df["month"] = df["date"].astype("datetime64[ns]").dt.month
        
        if verbose is True:
            print("Loaded DataFrame has the following columns:")
            for col in df:
                print("Column \'" + col + "\' with type", type(df[col][0]), ", e.g.", df[col][0])
            print("df.head():")
            print(df.head())
        
        return df
    except:
        print("Error, failed to find or load OHLC data from file with name \'"
              + filename + "\'. Please provide well-formed CSV file with OHLC stock data")

In [353]:
df_MSFT = load_OHLC_data(MSFT_DATA)
# df_AAPL = load_OHLC_data(AAPL_DATA)
# df_GOOGL = load_OHLC_data(GOOGL_DATA)
# df_HP = load_OHLC_data(HP_DATA)
# df_IBM = load_OHLC_data(IBM_DATA)
# df_WU = load_OHLC_data(WU_DATA)
# df_XRX = load_OHLC_data(XRX_DATA)

## 1.2 | Extract Features
- Common metrics for technical analysis are calculated to be later used as features
- Source for technical indicators: TA-lib, https://www.quantopian.com/posts/technical-analysis-indicators-without-talib-code

In [319]:
def extract_OHLC_features(df, time_horizons=TIME_HORIZONS):
    """
    Extract common technical stock analysis features from given OHLC stock data for distinct time horizons
    """
    # Calculate technical features for each time horizon
    for horizon in time_horizons:
#         # Future return of stock over given time horizon (e.g. this week's Monday to next week's Monday)
#         df["return_future_" + str(horizon) + "d"] = (df["close"].shift(-1*horizon)/df["close"])-1
        
        # Past return of stock over given time horizon (e.g. last week's Monday to this week's Monday)
        df["return_past_" + str(horizon) + "d"] = (df["close"].shift(horizon)/df["close"])-1
        
        # Implied volatility measured by standard deviation
        df["volatility_" + str(horizon) + "d"] = df["close"].rolling(horizon).std()
        
        # Moving averages (ma)
        df["ma_" + str(horizon) + "d"] = df["close"].rolling(horizon).mean()
        
#         Exponentially-weighted moving average (ewma)
#         df["ewma_" + str(horizon) + "d"] = pd.ewma(df["close"], span=horizon, min_periods=horizon-1)
#         df["ewma_" + str(horizon) + "d"] = df["close"].ewm(span=horizon, min_periods=horizon-1)
        
        # Momentum (absolute change in price over past horizon)
        df["momentum_" + str(horizon) + "d"] = df["close"].diff(horizon)
        
        # Rate of change during horizon period
        df["rateofchange_" + str(horizon) + "d"] = (df["close"].diff(horizon-1)) / (df["close"].shift(horizon-1))
        
#         Bollinger Bands
#         df["bollingerbands1_" + str(horizon) + "d"] = 4*df["volatility_" + str(horizon) + "d"] / df["ma_" + str(horizon) + "d"]
#         df["bollingerbands2_" + str(horizon) + "d"] = (df["close"] - df["ma_" + str(horizon) + "d"] + 2*df["volatility_" + str(horizon) + "d"]) / 4*df["volatility_" + str(horizon) + "d"]
        
        # TBD add other talib indicators from #Pivot Points, Supports and Resistances
    
    # OHLC average is used for stock price average of a given day
    df["ohlc_avg"] = df[["open", "high", "low", "close"]].mean(axis=1)
    
    # Replace NaNs with zeroes
    df = df.fillna(value=0)
    return df


print("#Features before extraction:", len(df_MSFT.columns))
df_MSFT = extract_OHLC_features(df_MSFT, TIME_HORIZONS)
print("#Features after extraction:", len(df_MSFT.columns))
df_AAPL = extract_OHLC_features(df_AAPL, TIME_HORIZONS)
df_GOOGL = extract_OHLC_features(df_GOOGL, TIME_HORIZONS)
df_HP = extract_OHLC_features(df_HP, TIME_HORIZONS)
df_IBM = extract_OHLC_features(df_IBM, TIME_HORIZONS)
df_WU = extract_OHLC_features(df_WU, TIME_HORIZONS)
df_XRX = extract_OHLC_features(df_XRX, TIME_HORIZONS)

if VERBOSE is True:
    print("With extracted features, dfMSFT.head() now yields following format:")
    print(df_MSFT.head())

#Features before extraction: 45
#Features after extraction: 45


## 1.3 | Anomaly Detection
- Anomaly defined as: ABS(return_past_1d) > threshold=5% (default)
- Such anomalies (5% threshold) occur in about 1.4% of instances for seven tech stock datasets

In [114]:
def detect_anomalies(df, verbose=VERBOSE, threshold=0.05):
    """
    Iterates through the DataFrame and prints out all dates where 1-day-return is greater than threshold=5% (default)
    """
    if verbose is True:
        print("Detecting anomalies where abs(1-day-return)>" + str(threshold*100) + " % for " + df["Name"][0])
    for i in range(len(df)):
        x = df["return_past_1d"][i]
        d = df["date"][i]
        if (abs(x) > threshold):
            global anomaly_counter
            anomaly_counter = anomaly_counter + 1
            if verbose is True:
                print("Anomaly: 1-day-return of " + str(round(x * 100, 2)) + "% on " + d.strftime("%A, %d.%m.%Y"))


anomaly_counter = 0
detect_anomalies(df_MSFT)
detect_anomalies(df_AAPL)
detect_anomalies(df_GOOGL)
detect_anomalies(df_HP)
detect_anomalies(df_IBM)
detect_anomalies(df_WU)
detect_anomalies(df_XRX)

print("anomaly_counter=" + str(counter) + ", or " + str(round(counter*100/(7*len(df_MSFT)), 2)) + "% of instances")

anomaly_counter=124, or 1.41% of instances


## 1.4 | Define classes
- This notebook evaluates DT and RF for stock recommendation (application no. 2 in thesis)
- Classes are defined for each time horizon to enable for later comparisons
- Base columns, on which classes are built, are removed to prevent illegal future-peeking features

In [331]:
def define_classes(df, time_horizons=TIME_HORIZONS):
    """
    Create target column in df: 1 means 'Yes, investor should buy stock', 0 means 'No, investor should not buy stock'.
    The assumed trading strategy here is, that the investor buy the stock on a given date and sells it after the horizon period.
    Also removes illegal (future-peeking) columns
    """
    for horizon in time_horizons:
        base_column_name = "return_future_" + str(horizon) + "d"
        class_name = "class_" + str(horizon) + "d"
        
        if class_name not in df.columns:
            df[class_name] = np.where(df[base_column_name] > 0, 1, 0)
            # Remove base column as it would be an illegal (future-peeking) feature
        if base_column_name in df.columns:
            df = df.drop(columns=[base_column_name])
    return df


df_MSFT = define_classes(df_MSFT)
df_AAPL = define_classes(df_AAPL)
df_GOOGL = define_classes(df_GOOGL)
df_HP = define_classes(df_HP)
df_IBM = define_classes(df_IBM)
df_WU = define_classes(df_WU)
df_XRX = define_classes(df_XRX)

print("classes (class_<horizon>d) created, base columns (return_future_<horizon>d) removed")

classes (class_<horizon>d) created, base columns (return_future_<horizon>d) removed


## 1.5 | Check class balance
- The two classes Yes (1) and No (0) should be balanced, else the evaluation technique must be adapted

In [158]:
def check_class_balance(df=df_MSFT, time_horizons=TIME_HORIZONS, verbose=VERBOSE, save_fig=SAVE_FIG):
    for horizon in time_horizons:
        class_name = "class_" + str(horizon) + "d"
        if verbose is True:
            print(df[class_name].value_counts())

        fig = plt.figure()
        df[class_name].hist()
        plt.xlabel("Class Value")
        plt.ylabel("Frequency")
        plt.title(df["Name"][0] + "-Class Value Histogram-" + str(horizon) + "d")
        if save_fig is True:
            plt.savefig("./Plots/Class-Balance-Check/" + df["Name"][0] + "-Class-Balance-Histogram-" + str(horizon) + "d.jpeg")
#         plt.close(fig) # clean memory if needed


check_class_balance(df_MSFT)
check_class_balance(df_AAPL)
check_class_balance(df_GOOGL)
check_class_balance(df_HP)
check_class_balance(df_IBM)
check_class_balance(df_WU)
check_class_balance(df_XRX)

##  TBD: 1.6 | Seaborn feature correlation plots

- Results: TBD
- Test old: Ergebnis: volatility-volume stark pos. korreliert (0.45), ohlc_avg-volume mäßig neg. korreliert (-0.36)

In [164]:
# def plot_corr_sns(df):
#     df = df.drop(columns=["open", "high", "low", "close", "Name"])
#     corr = df.corr()
    
#     plt.figure()
#     f, ax = plt.subplots(figsize=(5, 4)) #PARAM: figsize=(15, 12)
#     ax.set_title("Feature Correlation Matrix")
#     sns.heatmap(corr, cmap=plt.cm.Blues, mask=np.zeros_like(corr, dtype=np.bool), square=True, ax=ax)
# #     # plt.savefig("./plots/DataPreparation_MSFT-Stock-Data_correlation-matrix_v4.jpeg")
    
# #     plt.figure()
# #     sns.relplot(x="volume", y="volatility_3d", data=df);
    
# #     plt.figure()
# #     sns.relplot(x="volume", y="daily_return", data=df);
    
# #     plt.figure()
# #     sns.relplot(x="ohlc_avg", y="ma_3", data=df);


# plot_corr_sns(df_MSFT)
# # Add other df's

# 2 | Classification Stage
- Apply classifiers on datasets (for each time horizon-company combination)
- Plot findings

## 2.1 | Define training and test sets
- Seed RandomState to make algorithms reproducible

In [369]:
def train_test_split_data(df, train_size=0.7, time_horizons=TIME_HORIZONS, verbose=VERBOSE):
    """
    Generates training and testing set from a given DataFrame dataset.
    Assumes last COUNT(time_horizons) column(s) in DataFrame are classes,
    others are features.
    Returns features (X) and targets (y) in training- and testing sets.
    """
    # Remove non-numerical features for .fit() to work
    df = df.drop(columns=["Name", "date"])
    
    # Split DataFrame into features (X) and target (y)
    X = df.iloc[:,:-1*len(time_horizons)]
    y = df.iloc[:,-1*len(time_horizons):]
    
    # Use first (in same chronological order as time series) 70% to train and last 30% to test
    split_index = int(len(X) * train_size)
    if verbose is True:
        print("Split ist bei Index " + str(split_index) + " von " + str(len(X)))
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    return X_train, X_test, y_train, y_test

In [354]:
X_MSFT_train, X_MSFT_test, y_MSFT_train, y_MSFT_test = train_test_split_data(df_MSFT)
# Add other df's
# Better idea: just call this function later in evaluation, no extra variables needed

# 2.2 Hyperparameter tuning

# 3 | Evaluation Stage
- Build confusion matrixes and calculate performance metrics 
- Plot findings

## 3.1 | Measure accuracy by TimeSeriesSplit
- Data is in time series format, hence special time sseries cross validation method is more adequate than traditional k-fold cross validation

In [ ]:
# TBD: from sklearn.model_selection import TimeSeriesSplit

## 3.2 | Measure accuracy for given classifier and given X-s & y-s

In [368]:
def evaluate_accuracy(clf, X_train, X_test, y_train, y_test, verbose=VERBOSE):
    """
    Evaluates accuracy of given classifier for given training and test data.
    Only accepts y-DataFrames with one single column (as only one class is predicted)
    """
    if not isinstance(y_train, pd.Series) or not isinstance(y_test, pd.Series):
        print("y-DataFrames must have 1 column only.")
        return -1
    
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    if verbose is True:
        print("model=" + type(clf) + ", class=" + y_train.name + ", acc=" + str(round(acc, 2)))
    return acc

## 3.3 | Build confusion matrixes

In [ ]:
# TBD: Confusion Matrix

# 4 | Putting it all together (all above methods are called here in one single place)
- Prepare, classify and evaluate datasets

## 4.1 | Summarizing function for loading & extracting vs not data & splitting for train vs test

In [367]:
def generate_train_test_data(filename=MSFT_DATA, extract_features=False):
    """
    Serves as a one-stop-shop function for data loading incl. preparation so that classifier only relies on this single fucntion (and not many single functions) 
    """
    # Load data from .CSV (1.1)
    df = load_OHLC_data(filename=filename)
    
    # Extract features if applicable (1.2)
    if extract_features is True:
        df = extract_OHLC_features(df)
    
    # Detect anomalies (1.3) skipped
    # Define classes (1.4)
    df = define_classes(df)
    
    # Check class balance (1.5) skipped
    # Split data into training and testing samples
    X_train, X_test, y_train, y_test = train_test_split_data(df)
    
    return X_train, X_test, y_train, y_test

In [356]:
X_train, X_test, y_train, y_test = generate_train_test_data(extract_features=False)
print(X_train.columns)
print(y_train.columns)
X_train, X_test, y_train, y_test = generate_train_test_data(extract_features=True)
print(X_train.columns)
print(y_train.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'month'], dtype='object')
Index(['class_1d', 'class_5d', 'class_10d', 'class_30d', 'class_100d',
       'class_365d'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'month', 'return_past_1d',
       'volatility_1d', 'ma_1d', 'momentum_1d', 'rateofchange_1d',
       'return_past_5d', 'volatility_5d', 'ma_5d', 'momentum_5d',
       'rateofchange_5d', 'return_past_10d', 'volatility_10d', 'ma_10d',
       'momentum_10d', 'rateofchange_10d', 'return_past_30d', 'volatility_30d',
       'ma_30d', 'momentum_30d', 'rateofchange_30d', 'return_past_100d',
       'volatility_100d', 'ma_100d', 'momentum_100d', 'rateofchange_100d',
       'return_past_365d', 'volatility_365d', 'ma_365d', 'momentum_365d',
       'rateofchange_365d', 'ohlc_avg'],
      dtype='object')
Index(['class_1d', 'class_5d', 'class_10d', 'class_30d', 'class_100d',
       'class_365d'],
      dtype='object')


### TBD: Compare INITIAL DF (call fucntion) with EXTRACTED DF (cal lfunction) to see if engineering any good

## 4.2 | Apply & evaluate different classifiers on time horizons

In [390]:
def evaluate_models(dataset=MSFT_DATA, clf=DummyClassifier(random_state=42), extract_features=False, time_horizon=2, verbose=VERBOSE):
    """
    Evaluates gives  classifier on given data with or without extracted features for given time horizon
    """
    # Load data in "ready-to-classify" form with one-stop-shop function (4.1)
    X_train, X_test, y_train, y_test = generate_train_test_data(dataset, extract_features)
    
    # Evaluate classifier on data
    acc = evaluate_accuracy(clf, X_train, X_test, y_train[y_train.columns[time_horizon]], y_test[y_test.columns[time_horizon]])
    if verbose is True:
        print("clf=" + str(type(clf)), "dataset=" + dataset + " extracteds=" + str(extract_features) + ", time_horizon=" + str(time_horizon) + ", acc=" + str(round(acc, 2)))
    return acc

In [391]:
evaluate_models()

0.544973544973545

## 4.3 | Final master function
- Varying 4-5 dimensions too complex, hence the time horizon is fixed in the master function
- TBD: Toggle Hyperparameter Tuning somewhere
- TBD: Correct testing for 0 values at the end, e.g. 365 horizon return in last week's data is always 0!! Must be accounted for

In [392]:
# TIME_HORIZONS = [1, 5, 10, 30, 100, 365] <- use index from 0 (1 day) to 5 (365 days) to determine time horizon
def master_function(time_horizon=2):
    # Create classifiers that are to be evaluated
    clf1 = DummyClassifier(random_state=42)
    clf2 = DecisionTreeClassifier(random_state=42)
    clf3 = RandomForestClassifier(n_estimators=10, random_state=42)
    clfs = [clf1, clf2, clf3]
    
    # Iterate over models & datasets to evaluate classifiers on each dataset
    for clf in clfs:
        cum_acc = 0
        for dataset_path in TECH_GROUP:
            # Load data and evaluate all models on it
            evaluate_models(dataset=dataset_path, extract_features=False, time_horizon=time_horizon)
            cum_acc = cum_acc + evaluate_models(dataset=dataset_path, extract_features=True, time_horizon=time_horizon)
        clf_avg_acc = cum_acc/len(TECH_GROUP)
        print("avg acc for " + str(type(clf)) + " is " + str(round(clf_avg_acc, 2)))

In [393]:
master_function()

avg acc for <class 'sklearn.dummy.DummyClassifier'> is 0.52
avg acc for <class 'sklearn.tree.tree.DecisionTreeClassifier'> is 0.52
avg acc for <class 'sklearn.ensemble.forest.RandomForestClassifier'> is 0.52


# Comparison table: Model-Extractets-Horizon